# Simple examples

## Using threads

In [ ]:
import threading
import time
import sys

a = ''

def task_1():
    global a
    for i in range(10):
        print('o', end='')
        sys.stdout.flush()
        a += 'o'
        print(a)
        time.sleep(1)
        
def task_2():
    global a
    for i in range(20):
        print('O', end='')
        sys.stdout.flush()
        a += 'O'
        print(a)
        time.sleep(0.6)
        
thread_1 = threading.Thread(target=task_1)
thread_2 = threading.Thread(target=task_2)

thread_1.start()
thread_2.start()
print("(Both threads have started)")

thread_1.join() # Wait for thread_1 to finish
thread_2.join()
print("\nBoth threads have finished")

print(a)

 Notice that all threads share the same process memory space.

## Using processes

In [ ]:
# This code does not work!

import multiprocessing
import time
import sys

a = ''

def task_1():
    global a
    for i in range(10):
        print('o', end='')
        sys.stdout.flush()
        a += 'o'
        print(a)
        time.sleep(1)
        
def task_2():
    global a
    for i in range(20):
        print('O', end='')
        sys.stdout.flush()
        a += 'O'
        print(a)
        time.sleep(0.6)
        
process_1 = multiprocessing.Process(target=task_1)
process_2 = multiprocessing.Process(target=task_2)

process_1.start()
process_2.start()
print("(Both processes have started)")

process_1.join()
process_2.join()
print("\nBoth processes have finished")

print(a)

### But ... why `a` has not been modified? Why the processed do not share `a`?

By definition, processes must [__fork__](https://en.wikipedia.org/wiki/Fork_(system_call) (make a copy of itself, that is, the code and the used memory) before start running. In the previous example, the Python interpreter forks twice and the two childs are run in parallel while the parent process waits for their completition. Neither, the child processes nor the parent process share their global state (where `a` is defined).

### 2.1. How to share data between processes?

There are several options. One of them is to use a [shared memory `Manager()`](https://docs.python.org/3/library/multiprocessing.html#sharing-state-between-processes):

In [ ]:
import multiprocessing
import time
import sys
import ctypes

def task_1(a):
    for i in range(10):
        print('o', end='')
        sys.stdout.flush()
        a.value += 'o'
        time.sleep(1)
        
def task_2(a):
    for i in range(20):
        print('O', end='')
        sys.stdout.flush()
        a.value += 'O'
        time.sleep(0.6)

manager = multiprocessing.Manager()
# See https://docs.python.org/3/library/ctypes.html#module-ctypes
a = manager.Value(ctypes.c_char_p, "")    

process_1 = multiprocessing.Process(target=task_1, args=(a,))
process_2 = multiprocessing.Process(target=task_2, args=(a,))

process_1.start()
process_2.start()
print("(Both processes have started)")

process_1.join()
process_2.join()
print("\nBoth processes have finished")

print(a.value)

## Using corutines

In [1]:
def producer(sentence, next_coroutine): 
    ''' 
    Producer which just split strings and 
    feed it to pattern_filter coroutine 
    '''
    tokens = sentence.split(" ") 
    for token in tokens: 
        next_coroutine.send(token) 
    next_coroutine.close() 
  
def pattern_filter(pattern="ing", next_coroutine=None): 
    ''' 
    Search for pattern in received token  
    and if pattern got matched, send it to 
    print_token() coroutine for printing 
    '''
    print("Searching for {}".format(pattern)) 
    try: 
        while True: 
            token = (yield) 
            if pattern in token: 
                next_coroutine.send(token) 
    except GeneratorExit: 
        print("Done with filtering!!") 
  
def print_token(): 
    ''' 
    Act as a sink, simply print the 
    received tokens 
    '''
    print("I'm sink, i'll print tokens") 
    try: 
        while True: 
            token = (yield) 
            print(token) 
    except GeneratorExit: 
        print("Done with printing!") 
  
pt = print_token() 
pt.__next__() 
pf = pattern_filter(next_coroutine = pt) 
pf.__next__() 
  
sentence = "Bob is running behind a fast moving car"
producer(sentence, pf)

I'm sink, i'll print tokens
Searching for ing
running
moving
Done with filtering!!
